In [1]:
%run /spark-data/CRM/utilities/common_utility.ipynb

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/13 06:20:48 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Initialize Spark Session

In [2]:
spark = initialize_spark_session("Products Cleaning")

24/09/13 06:20:49 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


# Logs Configuration

In [3]:
log_file_path = 'logs/products_cleaning.log'
logger = initialize_logger(log_file_path)

logger.info("Logger initialized with dynamic path!")

2024-09-13 06:20:49,987 - logger - INFO - Logger initialized with dynamic path!


# Dataset Load

In [4]:
products_file_path = "/spark-data/CRM/Dataset/products.csv"
products_df = load_data_files(products_file_path)
display_dataframes(products_df)

2024-09-13 06:20:52,779 - logger - INFO - Displayed first 5 records of Spark DataFrame.


+----------+--------------+-----------+------+
|Product_ID|Product_Name  |Category   |Price |
+----------+--------------+-----------+------+
|1         |Sofa Set      |Home       |411.0 |
|2         |Laptop        |Electronics|333.0 |
|3         |Dining Table  |Home       |645.0 |
|4         |Vacuum Cleaner|NULL       |290.0 |
|5         |Mobile Phone  |Electronics|1738.0|
+----------+--------------+-----------+------+
only showing top 5 rows



# Data Preprocessing

In [5]:
# Step 1: Count missing (null) values for each column before filling them
logger.info("Step 1: Counting missing values in each column before filling them...")
missing_values_before = count_missing_values(products_df)
missing_values_before.show()

# Step 2: Count duplicates before dropping them
logger.info("Step 2: Checking for duplicates in each column before dropping them...")
duplicate_count = count_duplicates_per_column(products_df)
duplicate_count.show()

# Step 3: Drop duplicates based on 'Customer_ID' if any are found
logger.info("Step 3: Checking and removing duplicate records based on 'Customer_ID'...")
cleaned_products_df = drop_duplicates(products_df, "Product_ID")

# Step 4: Fill missing values in 'Email' and 'Phone' columns
logger.info("Step 4: Filling missing values in 'Email' and 'Phone' columns...")
cleaned_products_df = fill_missing_values(cleaned_products_df, {"Category": "Uncategorized"})
cleaned_products_df.show(5, truncate=False)

# Step 5: Capitalize the first letter of the first and last names in the 'Name' and 'Country' columns
logger.info("Step 5: Capitalizing the first letter of each word in the 'Product_Name' and 'Category' columns...")
cleaned_products_df = capitalize_columns(cleaned_products_df, ["Product_Name", "Category"])
print("Completed.")

# Step 6: Cross-validation - Count missing values again after filling them
logger.info("Step 6: Counting missing values in each column after filling them...")
missing_values_after = count_missing_values(cleaned_products_df)
missing_values_after.show()

# Step 7: Count duplicates after dropping them
logger.info("Step 7: Checking for duplicates in each column after dropping them...")
duplicate_count_after = count_duplicates_per_column(cleaned_products_df)
duplicate_count_after.show()

# Step 8: Handle negative or zero prices by replacing with average price
logger.info("Step 8: Replacing negative or zero prices with the average price...")
avg_price = products_df.agg({"Price": "avg"}).collect()[0][0]
cleaned_products_df = cleaned_products_df.withColumn("Price", when(col("Price") <= 0, avg_price).otherwise(col("Price")))
print("completed.")

# Step 9: Export the cleaned data to a CSV file
logger.info("Step 9: Exporting the cleaned data to 'cleaned_products.csv'...")
save_df_to_csv(cleaned_products_df, "/spark-data/CRM/cleaned_data/cleaned_products.csv")

# show duplicates
# get_duplicate_data_per_column(cleaned_products_df)

# Display the count of records after phone number processing
record_count_after_cleaning = cleaned_products_df.count()
print(f"Number of records after cleaning: {record_count_after_cleaning}")

logger.info("Data cleaning and export completed successfully.")

2024-09-13 06:20:52,794 - logger - INFO - Step 1: Counting missing values in each column before filling them...
2024-09-13 06:20:53,353 - logger - INFO - Step 2: Checking for duplicates in each column before dropping them...


+----------+------------+--------+-----+
|Product_ID|Product_Name|Category|Price|
+----------+------------+--------+-----+
|         0|           0|       3|    0|
+----------+------------+--------+-----+



2024-09-13 06:20:55,968 - logger - INFO - Step 3: Checking and removing duplicate records based on 'Customer_ID'...


+------------+---------------+
|      Column|Duplicate_Count|
+------------+---------------+
|  Product_ID|              1|
|Product_Name|              1|
|    Category|              5|
|       Price|              2|
+------------+---------------+



2024-09-13 06:20:56,214 - logger - INFO - Number of duplicate records before dropping: 1
2024-09-13 06:20:56,557 - logger - INFO - Number of duplicate records after dropping: 0
2024-09-13 06:20:56,558 - logger - INFO - Step 4: Filling missing values in 'Email' and 'Phone' columns...
2024-09-13 06:20:56,942 - logger - INFO - Step 5: Capitalizing the first letter of each word in the 'Product_Name' and 'Category' columns...
2024-09-13 06:20:56,973 - logger - INFO - Step 6: Counting missing values in each column after filling them...


...

+----------+------------+--------+-----+
|Product_ID|Product_Name|Category|Price|
+----------+------------+--------+-----+
|         0|           0|       0|    0|
+----------+------------+--------+-----+



2024-09-13 06:20:59,088 - logger - INFO - Step 8: Replacing negative or zero prices with the average price...
2024-09-13 06:20:59,245 - logger - INFO - Step 9: Exporting the cleaned data to 'cleaned_products.csv'...


+------------+---------------+
|      Column|Duplicate_Count|
+------------+---------------+
|  Product_ID|              0|
|Product_Name|              0|
|    Category|              5|
|       Price|              1|
+------------+---------------+

completed.


2024-09-13 06:20:59,627 - logger - INFO - Data cleaning and export completed successfully.


Number of records after cleaning: 50


In [6]:
spark.stop()